In diesem Notebook wird eine Delayed Streaming Klassifizierung für t_air mit V = 7000 durchgeführt

In [1]:
import numpy as np
import pandas as pd

from skmultiflow.trees import HoeffdingTreeClassifier
from skmultiflow.trees import HoeffdingAdaptiveTreeClassifier
from skmultiflow.trees import ExtremelyFastDecisionTreeClassifier
from skmultiflow.meta import AdaptiveRandomForestClassifier
from skmultiflow.bayes import NaiveBayes
from skmultiflow.meta import AccuracyWeightedEnsembleClassifier
from skmultiflow.lazy import KNNADWINClassifier

from skmultiflow.data import TemporalDataStream

from skmultiflow.evaluation import EvaluatePrequentialDelayed

In [3]:
import warnings
warnings.filterwarnings('ignore')

#set columns
attribute = ["Airline","Flight","AirportFrom","AirportTo","DayOfWeek","Time","Length"]
target = "Delay"
timestamp = "datetime"
nominal=['Airline', 'Flight', 'AirportFrom', 'AirportTo', 'DayOfWeek']

#read csv
t_air = pd.read_csv("data/t_air.csv")

#convert time to datetime
t_air[timestamp] = pd.to_datetime(t_air[timestamp])

#get X, y and time
X = t_air[attribute].values
y = t_air[target].values
time = t_air[timestamp].values

#set a delay
delay_time = 7000

#create stream
t_air_stream = TemporalDataStream(X, y, time, sample_delay=delay_time, ordered=False)

#set classifier
ht = HoeffdingTreeClassifier(leaf_prediction='nb', nominal_attributes=nominal)
hat = HoeffdingAdaptiveTreeClassifier(nominal_attributes=nominal)
efdt = ExtremelyFastDecisionTreeClassifier(nominal_attributes=nominal)
arf = AdaptiveRandomForestClassifier(nominal_attributes=nominal)
snb = NaiveBayes(nominal_attributes=nominal)
awe = AccuracyWeightedEnsembleClassifier(n_estimators=15, base_estimator=NaiveBayes(nominal_attributes=nominal))
knn_adwin = KNNADWINClassifier(n_neighbors=10)

#set evaluator
evaluator = EvaluatePrequentialDelayed(n_wait=10,
                                pretrain_size=100,
                                max_samples=X.shape[0],
                                output_file='results_air_t.csv',
                                metrics=['accuracy', 'kappa', 
                                         'precision', 'recall', 
                                         'f1', 'running_time', 
                                         'model_size'])

#evaluate
evaluator.evaluate(stream=t_air_stream, 
                   model=[ht, hat, efdt, arf, 
                          snb, awe, knn_adwin], 
                   model_names=['Hoeffding Tree', 'Hoeffding Adaptive Tree', 
                                'Extremely Fast Decision Tree', 
                                'Adaptive Random Forest', 
                                'Naive Bayes', 'Accuracy Weighted Ensembler', 'KNN ADWIN' ])

Prequential Evaluation Delayed
Evaluating 1 target(s).
Pre-training on 100 sample(s).
Evaluating...
 ###################- [95%] [351.19s]Processed samples: 10000
Mean performance:
Hoeffding Tree - Accuracy     : 0.5937
Hoeffding Tree - Kappa        : 0.1686
Hoeffding Tree - Precision: 0.5109
Hoeffding Tree - Recall: 0.5321
Hoeffding Tree - F1 score: 0.5213
Hoeffding Tree - Training time (s)  : 1.52
Hoeffding Tree - Testing time  (s)  : 1.56
Hoeffding Tree - Total time    (s)  : 3.09
Hoeffding Tree - Size (kB)          : 52.4365
Hoeffding Adaptive Tree - Accuracy     : 0.5866
Hoeffding Adaptive Tree - Kappa        : 0.0942
Hoeffding Adaptive Tree - Precision: 0.5048
Hoeffding Adaptive Tree - Recall: 0.2930
Hoeffding Adaptive Tree - F1 score: 0.3708
Hoeffding Adaptive Tree - Training time (s)  : 7.02
Hoeffding Adaptive Tree - Testing time  (s)  : 1.18
Hoeffding Adaptive Tree - Total time    (s)  : 8.20
Hoeffding Adaptive Tree - Size (kB)          : 60.5322
Extremely Fast Decision Tree - 

[HoeffdingTreeClassifier(binary_split=False, grace_period=200,
                         leaf_prediction='nb', max_byte_size=33554432,
                         memory_estimate_period=1000000, nb_threshold=0,
                         no_preprune=False,
                         nominal_attributes=['Airline', 'Flight', 'AirportFrom',
                                             'AirportTo', 'DayOfWeek'],
                         remove_poor_atts=False, split_confidence=1e-07,
                         split_criterion='info_gain', stop_mem_management=False,
                         tie_threshold=0.05),
 HoeffdingAdaptiveTreeClassifier(binary_split=False, bootstrap_sampling=True,
                                 grace_period=200, leaf_prediction='nba',
                                 max_byte_size=33554432,
                                 memory_estimate_period=1000000, nb_threshold=0,
                                 no_preprune=False,
                                 nominal_attributes=['